# Table of Contents
<a id="table-of-contents"></a>
- [1 What is TrueFoundry?](#1)
- [2 Spaceship Titanic](#2)
- [3 Preparation](#3)
    - [3.1 Essential Packages](#3.1)
    - [3.2 TrueFoundry Packages & Login](#3.2)
- [4 Data Loading and Preprocessing](#4)
- [5 Models](#5)
    - [5.1 XGBoost Classifier](#5.1)
    - [5.2 LGBM Classifier](#5.2)
    - [5.3 Catboost Classifier](#5.3)
- [6 TrueFoundry Results](#6)
    - [6.1 Projects](#6.1)
    - [6.2 Runs](#6.2)
        - [6.2.1 Overview](#6.2.1)
        - [6.2.2 Run Metrics](#6.2.2)
        - [6.2.3 Data & Feature Metrics](#6.2.3)
        - [6.2.4 General Artifact](#6.2.4)
    - [6.3 Models Comparison](#6.3)

[back to top](#table-of-contents)
<a id="1"></a>
# 1 What is TrueFoundry? 

Truefoundry aims to provide the different components in a Machine learning stack - all wound together in a way that they talk to each other seamlessly and teams don't have to spend time glueing pieces together. While all the pieces are knit tightly together, we also design all the components in a way that they can be seamlessly integrated with other tools in the future. TrueFoundry can be accessed at https://app.truefoundry.com/mlfoundry.

Truefoundry comprises of the following pieces to tie together things seamlessly:
1. `MlFoundry`: used during model training to log your model artifacts, parameters, data & code so as to be able to collaborate with your team and reproduce Machine Learning Experiments.
2. `ServiceFoundry`: single API which containerizes and deploys your model to a managed Kubernetes Cluster. This also generates a Grafana cluster with complete visibility of your Service Health, System Logs, and Kubernetes Workspace.
3. `Monitoring`: model input-output monitoring, data drift charts, and root-cause analysis when things break. Coming soon! 

[back to top](#table-of-contents)
<a id="2"></a>
# 2 Spaceship Titanic 
The Spaceship Titanic was an interstellar passenger liner launched a month ago. With almost 13,000 passengers on board, the vessel set out on its maiden voyage transporting emigrants from our solar system to three newly habitable exoplanets orbiting nearby stars.

While rounding Alpha Centauri en route to its first destination—the torrid 55 Cancri E—the unwary Spaceship Titanic collided with a spacetime anomaly hidden within a dust cloud. Sadly, it met a similar fate as its namesake from 1000 years before. Though the ship stayed intact, almost half of the passengers were transported to an alternate dimension!

To help rescue crews and retrieve the lost passengers, you are challenged to predict which passengers were transported by the anomaly using records recovered from the spaceship’s damaged computer system.

Help save them and change history!

[back to top](#table-of-contents)
<a id="3"></a>
# 3 Preparation
Prepare packages and data that will be used in the analysis process and we will use `TrueFoundry` to track our experiments and `Essential packages` that will be loaded are mainly for data manipulation, data visualization and modeling. There are 2 datasets that are used in the analysis, they are train and test dataset. The main use of train dataset is to train models and use it to predict test dataset. While sample submission file is used to informed participants on the expected submission for the competition.

<a id="3.1"></a>
## 3.1 Essential Packages

In [ ]:
import pandas as pd
import numpy as np
import sklearn
from IPython.display import Image
from sklearn import preprocessing
from sklearn.metrics import accuracy_score
from sklearn.model_selection import StratifiedKFold
from sklearn.linear_model import LinearRegression
from scipy.stats import boxcox
from xgboost import XGBClassifier 
from catboost import CatBoostClassifier
from lightgbm import LGBMClassifier

In [ ]:
from kaggle_secrets import UserSecretsClient
user_secrets = UserSecretsClient()
secret_value_0 = user_secrets.get_secret("api_key")

<a id="3.2"></a>
## 3.2 TrueFoundry Packages & Login
In the first step we will install `TrueFoundry` using `pip install mlfoundry` and import it is as `mlf`.

In [ ]:
!pip install mlfoundry
import mlfoundry as mlf

Login to `TrueFoundry` platform and get the API key. `Create New API keys` if we don't have it or copy the generated API keys before. API key can be found at https://app.truefoundry.com/settings.

In [ ]:
Image("../input/experimenttracking-truefoundry/API_Key.png")

We will login using `mlf.get_client` and use pass our API Key on `api_key`. We also create a `project name`.

In [ ]:
client = mlf.get_client(api_key=secret_value_0)
project_name = 's-titanic'

[back to top](#table-of-contents)
<a id="4"></a>
# 4 Data Loading and Preprocessing
There are 3 thing that will be done in this section:
1. Load `train` dataset.
2. Convert target variable `Transported`, from `True` and `False` to `1` and `0` using `label encoder`. 
3. `Categorical` features will also be converted using `label encoder`.

In [ ]:
folds = 5
train_load = pd.read_csv("../input/spaceship-titanic/train.csv")

features = [col for col in train_load.columns if col not in ['PassengerId', 'Transported', 'Name']]
cat_features = ['HomePlanet', 'CryoSleep', 'VIP', 'Destination', 'Cabin']
cont_features = ['Age', 'RoomService', 'FoodCourt', 'ShoppingMall', 'Spa', 'VRDeck']

# convert Transported into 0 and 1
le = preprocessing.LabelEncoder()
train_load['Transported'] = le.fit_transform(train_load['Transported'])

# convert categorical features
for feature in cat_features:
    le = preprocessing.LabelEncoder()
    train_load[feature] = le.fit_transform(train_load[feature])

[back to top](#table-of-contents)
<a id="5"></a>
# 5 Models
We are going to use 3 models: `XGBoost Classifier`, `Catboost Classifier` and `LGBM Classifier`. We will also create `5 folds` cross validation. We will not perform any hyperparameters tuning. 

**Observations:**
- It seems that `Catboost Classifer` has the best OOF score of `0.8029448981939492`.
- The second best performer is `LGBM Classifier` with OOF score of `0.8019095824226389`.
- `Catboost Classifer` and `LGBM Classifer` performance are really close. If we inspect in each folds, `Catboost Classifier` has beaten `LGBM Classifier` in fold `0, 1 and 2` while in fold `3 and 4` `LGBM Classifier` performed better than `Catboost Classifier`.

We can improve the results by performing `features engineering` and `hyperparameters tuning`.

**TrueFoundry Experiment Tracking**

We will track our experiment using `TrueFoundry`. Below are some explanations in the code related to `TrueFoundry` experiment tracking. We are using `xgboost classifier` as an example and consistently being used for other models:
- Create `run = client.create_run(project_name=project_name, run_name="xgboost")` to start logging our experiment by creating project name and the run name. In this case we log the project name as `s-titanic` that has been setup before and naming our run as `xgboost` for XGBoost Classifier model.
- We can track our dataset including `target prediction` and `target actual` using `run.log_dataset(features=train_df[features], dataset_name="full", actuals=train_df['Transported'], predictions=train_oof)`. Logging our `actual` and `prediction` target will help us to compare them in `TrueFoundry` platform. This line of codes are putted the end of the code as we will need to wait until all the prediction in each fold finished.   
- We will also do the same thing for each fold-dataset, we use`run.log_dataset(dataset_name="fold_"+str(fold), features=X_valid, actuals=y_valid, predictions=temp_oof)` but we will perform this after a fold prediction finished.
- To log hyperparameters from the model, we use `run.log_params(model.get_xgb_params())`. We can only log 1 set hyperparameters, that's why we put it at the end of the code.
- We can also log our validation accuracy metrics over time using below code:<br>
    `results = model.evals_result()`<br>
    `epochs = len(results['validation_0']['error'])`<br>
    `accuracy_fold = [1-err for err in results['validation_0']['error']]`<br>
    `for global_step in range(epochs):`<br>
    `run.log_metrics(metric_dict={f'Accuracy_fold_{fold}':accuracy_fold[global_step]}, step=global_step)`<br>
  We can compare the `accuracy` of each `fold` and `OOF` accuracy across all model over time.
- The last but not least, we need to end our run using `run.end()`.

It may be easier to understand after seeing the code in practice and a comment of `#TrueFoundry` at the end of the code.

[back to top](#table-of-contents)
<a id="5.1"></a>
## 5.1 XGBoost Classifier
Below are the results from XGBoost Classifier:

- Fold 0 Accuracy:  0.79700977573318
- Fold 1 Accuracy:  0.7786083956296722
- Fold 2 Accuracy:  0.7912593444508338
- Fold 3 Accuracy:  0.7957422324510932
- Fold 4 Accuracy:  0.7957422324510932
- OOF AUC:  0.7916714597952376

In [ ]:
run = client.create_run(project_name=project_name, run_name="xgboost") #TrueFoundry

# Load and Prep Data
train_df = train_load.copy()

train_oof = np.zeros((8693,))
skf = StratifiedKFold(n_splits=folds, shuffle=True, random_state=42)
for fold, (train_idx, valid_idx) in enumerate(skf.split(train_df[features], train_df['Transported'])):
    train, valid = train_df.iloc[train_idx], train_df.iloc[valid_idx]
    y_train = train['Transported']
    y_valid = valid['Transported']
    X_train = train[features]
    X_valid = valid[features]
    
    model = XGBClassifier(random_state=42, verbosity=0)
    model = model.fit(X_train, y_train, eval_metric="error", eval_set=[(X_valid, y_valid)], verbose=0)
    
    temp_oof = model.predict(X_valid)
    train_oof[valid_idx] = temp_oof    
    
    print(f'Fold {fold} Accuracy: ', accuracy_score(y_valid, temp_oof))
    
    results = model.evals_result()
    epochs = len(results['validation_0']['error'])
    accuracy_fold = [1-err for err in results['validation_0']['error']]
    for global_step in range(epochs):
        run.log_metrics(metric_dict={f'Accuracy_fold_{fold}':accuracy_fold[global_step]}, step=global_step) #TrueFoundry
    run.log_dataset(dataset_name="fold_"+str(fold), features=X_valid, actuals=y_valid, predictions=temp_oof) #TrueFoundry
    
accuracy_final = accuracy_score(train_df['Transported'], train_oof)    
print(f'OOF AUC: ', accuracy_final)

run.log_params(model.get_params()) #TrueFoundry
run.log_metrics(metric_dict={'Accuracy_OOF':accuracy_final}) #TrueFoundry
run.log_dataset(features=train_df[features], dataset_name="full", actuals=train_df['Transported'], predictions=train_oof) #TrueFoundry

run.end() #TrueFoundry

[back to top](#table-of-contents)
<a id="5.2"></a>
## 5.2 LGBM Classifier
Below are the results from LGBM Classifier:

- Fold 0 Accuracy:  0.8021851638872916
- Fold 1 Accuracy:  0.7906843013225991
- Fold 2 Accuracy:  0.8062104657849338
- Fold 3 Accuracy:  0.8072497123130035
- Fold 4 Accuracy:  0.8032220943613348
- OOF AUC:  0.8019095824226389

In [ ]:
run = client.create_run(project_name=project_name, run_name="lgbm") #TrueFoundry

# Load and Prep Data
train_df = train_load.copy()

train_oof = np.zeros((8693,))
skf = StratifiedKFold(n_splits=folds, shuffle=True, random_state=42)
for fold, (train_idx, valid_idx) in enumerate(skf.split(train_df[features], train_df['Transported'])):
    train, valid = train_df.iloc[train_idx], train_df.iloc[valid_idx]
    y_train = train['Transported']
    y_valid = valid['Transported']
    X_train = train[features]
    X_valid = valid[features]

    model = LGBMClassifier(random_state=42, verbosity=0)
    model = model.fit(X_train, y_train, eval_metric="binary_error", eval_set=[(X_valid, y_valid)], verbose=0)

    temp_oof = model.predict(X_valid)
    train_oof[valid_idx] = temp_oof
    
    print(f'Fold {fold} Accuracy: ', accuracy_score(y_valid, temp_oof))
    
    results = model.evals_result_
    epochs = len(results['valid_0']['binary_error'])
    accuracy_fold = [1-err for err in results['valid_0']['binary_error']]
    for global_step in range(epochs):
        run.log_metrics(metric_dict={f'Accuracy_fold_{fold}':accuracy_fold[global_step]}, step=global_step) #TrueFoundry
    run.log_dataset(dataset_name="fold_"+str(fold), features=X_valid, actuals=y_valid, predictions=temp_oof) #TrueFoundry
    
accuracy_final = accuracy_score(train_df['Transported'], train_oof)    
print(f'OOF AUC: ', accuracy_final)

run.log_params(model.get_params()) #TrueFoundry
run.log_metrics(metric_dict={'Accuracy_OOF':accuracy_final}) #TrueFoundry
run.log_dataset(features=train_df[features], dataset_name="full", actuals=train_df['Transported'], predictions=train_oof) #TrueFoundry

run.end() #TrueFoundry

[back to top](#table-of-contents)
<a id="5.3"></a>
## 5.3 Catboost Classifier
Below are the results from Catboost Classifier:

- Fold 0 Accuracy:  0.8113858539390454
- Fold 1 Accuracy:  0.8021851638872916
- Fold 2 Accuracy:  0.8102357676825762
- Fold 3 Accuracy:  0.7968929804372842
- Fold 4 Accuracy:  0.7940161104718066
- OOF AUC:  0.8029448981939492

In [ ]:
run = client.create_run(project_name=project_name, run_name="catboost") #TrueFoundry

# Load and Prep Data
train_df = train_load.copy()

train_oof = np.zeros((8693,))
skf = StratifiedKFold(n_splits=folds, shuffle=True, random_state=42)
for fold, (train_idx, valid_idx) in enumerate(skf.split(train_df[features], train_df['Transported'])):
    train, valid = train_df.iloc[train_idx], train_df.iloc[valid_idx]
    y_train = train['Transported']
    y_valid = valid['Transported']
    X_train = train[features]
    X_valid = valid[features]
    
    model = CatBoostClassifier(iterations=100, eval_metric='Accuracy', random_state=42)
    model = model.fit(X_train, y_train, eval_set=[(X_valid, y_valid)], use_best_model=False, verbose=0)
    
    temp_oof = model.predict(X_valid)
    train_oof[valid_idx] = temp_oof    
    
    print(f'Fold {fold} Accuracy: ', accuracy_score(y_valid, temp_oof))
    
    results = model.get_evals_result()
    epochs = len(results['validation']['Accuracy'])
    for global_step in range(epochs):
        run.log_metrics(metric_dict={f'Accuracy_fold_{fold}':results['validation']['Accuracy'][global_step]}, step=global_step) #TrueFoundry
    run.log_dataset(dataset_name="fold_"+str(fold), features=X_valid, actuals=y_valid, predictions=temp_oof) #TrueFoundry
    
accuracy_final = accuracy_score(train_df['Transported'], train_oof)    
print(f'OOF AUC: ', accuracy_final)

run.log_params(model.get_params()) #TrueFoundry
run.log_metrics(metric_dict={'Accuracy_OOF':accuracy_final}) #TrueFoundry
run.log_dataset(features=train_df[features], dataset_name="full", actuals=train_df['Transported'], predictions=train_oof) #TrueFoundry

run.end() #TrueFoundry

[back to top](#table-of-contents)
<a id="6"></a>
# 6 TrueFoundry Platform
In this section we will see how our `dataset`, `hyperparameters` and `metrics` have been logged in TrueFoundry. We will see into 2 sections: `Projects` and `Runs`.

[back to top](#table-of-contents)
<a id="6.1"></a>
## 6.1 Projects

We can check all of our projects in `ML Foundry` section. In this case we are looking for our `s-titanic` projects.

In [ ]:
Image("../input/experimenttracking-truefoundry/all_projects.png")

By clicking our project we will see our `Run`. We will see 3 runs that have been logged before which are `catboost`, `lgbm` and `xgboost`.

In [ ]:
Image("../input/experimenttracking-truefoundry/all_runs.png")

[back to top](#table-of-contents)
<a id="6.2"></a>
## 6.2 Runs
Let's check our `xgboost` run. In the upper side, we can see our `Run Name`, `Run Id`, `Author`, `Status`, `Last Updated On`, `Tags` and `Run Duration`. We can also put `tags` and `notes`.

<a id="6.2.1"></a>
### 6.2.1 Overview
- In the left side, we can see our log `Key Metrics` which is `accuracy`. It logs `fold_0` through `fold_4` including `OOF` accuracy metrics.
- In the right side, we can see our `hyperparameters` that has been logged. It also represents the latest hyperparameters and it would be the same accross the folds.

In [ ]:
Image("../input/experimenttracking-truefoundry/xgboost_overview.png")

<a id="6.2.2"></a>
### 6.2.2 Run Metrics
We can see each of our `validation` fold (0 through 4) accuracy in a line graph. Meaning we can see how it performs in each steps.

In [ ]:
Image("../input/experimenttracking-truefoundry/xgboost_run_metrics.png")

<a id="6.2.3"></a>
### 6.2.3 Data & Feature Metrics
We can see our folds dataset, in this case we can see `fold_0` through `fold_4` and `full` dataset as we have logged them before.
We can also see more details on each features by clicking the `Details`.

In [ ]:
Image("../input/experimenttracking-truefoundry/xgboost_data.png")

We can also see the comparsion between our predictions and actual performance for each fold (0 to 4) and also in the full dataset.

In [ ]:
Image("../input/experimenttracking-truefoundry/xgboost_data_2.png")

<a id="6.2.4"></a>
### 6.2.4 General Artifact
In here we can see our dataset that have been stored in `csv format` and can be re-downloaded.

In [ ]:
Image("../input/experimenttracking-truefoundry/xgboost_artifact.png")

[back to top](#table-of-contents)
<a id="6.3"></a>
## 6.3 Models Comparison
We can perform model comparison by clicking all the models that we want to compare. In this case we choose all of them.

In [ ]:
Image("../input/experimenttracking-truefoundry/models_comparison_1.png")

We can see the comparison of our models (catboost, lgbm and xgboost) in a line graph based on the `validation accuracy` metric for each fold. By hovering into the line, we can see the `accuracy` for each model steps in each fold.

In [ ]:
Image("../input/experimenttracking-truefoundry/models_comparison_2.png")